In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
pickle_file = '/Users/manishagalla/Documents/deepLearning/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [18]:
batch_size = 128
beta = 0.005
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
    weights = tf.Variable(
    tf.truncated_normal([1024,num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size , 1024]))
    biases_1 = tf.Variable(tf.zeros([1024]))
  
  # Training computation.
    l1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    l1 = tf.nn.relu(l1)
    l1_valid = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    l1_valid = tf.nn.relu(l1_valid)
    l1_test = tf.matmul(tf_test_dataset, weights_1) + biases_1
    l1_test = tf.nn.relu(l1_test)
    logits = tf.matmul(l1, weights) + biases
    logits_valid  = tf.matmul(l1_valid, weights) + biases
    logits_test = tf.matmul(l1_test, weights) + biases
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) 
    loss = tf.reduce_mean(loss + beta* tf.nn.l2_loss(weights))
  
  # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(logits_valid)
    test_prediction = tf.nn.softmax(logits_test)

In [ ]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

In [21]:
num_steps = 3001

train_dataset_new = train_dataset[:1000,:]
train_labels_new = train_labels[:1000]
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels_new.shape[0] - batch_size)
    # Generate a minibatch.
        batch_data = train_dataset_new[offset:(offset + batch_size), :]
        batch_labels = train_labels_new[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 427.783783
Minibatch accuracy: 10.9%
Validation accuracy: 29.5%
Minibatch loss at step 500: 1.818797
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Minibatch loss at step 1000: 0.149767
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 1500: 0.017639
Minibatch accuracy: 100.0%
Validation accuracy: 78.3%
Minibatch loss at step 2000: 1.578132
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 2500: 0.133938
Minibatch accuracy: 100.0%
Validation accuracy: 78.3%
Minibatch loss at step 3000: 0.017969
Minibatch accuracy: 100.0%
Validation accuracy: 78.7%
Test accuracy: 85.3%


In [23]:
batch_size = 128
beta = 0.005
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
    weights = tf.Variable(
    tf.truncated_normal([1024,num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size , 1024]))
    biases_1 = tf.Variable(tf.zeros([1024]))
    to_drop = tf.placeholder(tf.float32)
  
  # Training computation.
    l1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    l1 = tf.nn.relu(l1)
    drop_out = tf.nn.dropout(l1,to_drop)
    l1_valid = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    l1_valid = tf.nn.relu(l1_valid)
    l1_test = tf.matmul(tf_test_dataset, weights_1) + biases_1
    l1_test = tf.nn.relu(l1_test)
    logits = tf.matmul(drop_out, weights) + biases
    logits_valid  = tf.matmul(l1_valid, weights) + biases
    logits_test = tf.matmul(l1_test, weights) + biases
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) 
    loss = tf.reduce_mean(loss + beta* tf.nn.l2_loss(weights))
  
  # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(logits_valid)
    test_prediction = tf.nn.softmax(logits_test)

In [29]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,to_drop:1.5 }
        _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 349.460358
Minibatch accuracy: 8.6%
Validation accuracy: 31.0%
Minibatch loss at step 500: 24.632132
Minibatch accuracy: 78.9%
Validation accuracy: 75.8%
Minibatch loss at step 1000: 23.336401
Minibatch accuracy: 69.5%
Validation accuracy: 73.5%
Minibatch loss at step 1500: 25.747618
Minibatch accuracy: 64.8%
Validation accuracy: 71.8%
Minibatch loss at step 2000: 5.287636
Minibatch accuracy: 79.7%
Validation accuracy: 80.0%
Minibatch loss at step 2500: 10.065284
Minibatch accuracy: 71.1%
Validation accuracy: 77.1%
Minibatch loss at step 3000: 2.499735
Minibatch accuracy: 82.8%
Validation accuracy: 80.1%
Test accuracy: 86.9%


In [30]:
num_steps = 3001

train_dataset_new = train_dataset[:1000,:]
train_labels_new = train_labels[:1000]
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels_new.shape[0] - batch_size)
    # Generate a minibatch.
        batch_data = train_dataset_new[offset:(offset + batch_size), :]
        batch_labels = train_labels_new[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, to_drop:1.0}
        _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 309.402374
Minibatch accuracy: 11.7%
Validation accuracy: 35.2%
Minibatch loss at step 500: 1.687897
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Minibatch loss at step 1000: 0.138593
Minibatch accuracy: 100.0%
Validation accuracy: 78.3%
Minibatch loss at step 1500: 0.014909
Minibatch accuracy: 100.0%
Validation accuracy: 79.1%
Minibatch loss at step 2000: 0.007049
Minibatch accuracy: 100.0%
Validation accuracy: 79.7%
Minibatch loss at step 2500: 0.008507
Minibatch accuracy: 100.0%
Validation accuracy: 79.7%
Minibatch loss at step 3000: 0.006608
Minibatch accuracy: 100.0%
Validation accuracy: 79.5%
Test accuracy: 86.3%
